In [2]:
# Importing Libraries
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [1]:
# Raw dataset URLs
player_data_url <- "https://raw.githubusercontent.com/MohakB3/dsci100-project/refs/heads/main/data/players.csv"
sessions_data_url <- "https://raw.githubusercontent.com/MohakB3/dsci100-project/refs/heads/main/data/sessions.csv"

# Reading Data (with the column specification message disabled) and Omitting NAs
player_data <- na.omit(read_csv(player_data_url, show_col_types = FALSE))
sessions_data <- na.omit(read_csv(sessions_data_url, show_col_types = FALSE))

# Wrangling Sessions Data
sessions_wrangled_data <- sessions_data |>
    mutate(session_length = as.POSIXct(end_time,format = "%d/%m/%Y %H:%M") - as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")) |>
    select(hashedEmail, session_length) |>
    group_by(hashedEmail) |>
    summarize(player_average_session_length = as.numeric(mean(session_length)))

# Merging Player Data and Wrangled Sessions Data
merged_data <- left_join(sessions_wrangled_data, player_data, by = "hashedEmail") |> na.omit()

# Calculating Mean Values for all Quantitative Variables
mean_quantitative_data <- merged_data |>
    summarize(mean_age = mean(Age), mean_played_hours = mean(played_hours), mean_session_length = mean(player_average_session_length))
mean_quantitative_data

# Creating a Density Plot for Players' Total Time Played
options(repr.plot.width = 15, repr.plot.height = 5)
time_plot <- merged_data |>
    select(played_hours, subscribe) |>
    filter(played_hours >= 0, played_hours <= 20) |>
    ggplot(aes(x=played_hours, fill = subscribe, colour = subscribe)) + 
    geom_density(alpha = 0.5) +
    labs(x="Time Played (hours)",y="Density",fill = "Is Subscribed?", colour = "Is Subscribed?")

# Creating a Density Plot for Players' Age
options(repr.plot.width = 7, repr.plot.height = 5)
age_plot <- merged_data |>
    select(Age, subscribe) |>
    ggplot(aes(x=Age, fill = subscribe, colour = subscribe)) + 
    geom_density(alpha = 0.5) +
    labs(x="Player Age (years)",y="Density",fill = "Is Subscribed?", colour = "Is Subscribed?")

# Creating a Density Plot for Players' Average Session Length
session_length_plot <- merged_data |>
    select(player_average_session_length, subscribe) |>
    ggplot(aes(x=player_average_session_length, fill = subscribe, colour = subscribe)) + 
    geom_density(alpha = 0.5) +
    labs(x="Average Session Length (minutes)",y="Density",fill = "Is Subscribed?", colour = "Is Subscribed?")

# Rendering all Plots
time_plot
age_plot
session_length_plot
#head(player_data)
head(sessions_data)
head(merged_data)

ERROR: Error in read_csv(player_data_url, show_col_types = FALSE): could not find function "read_csv"


# Data Description
The original source contains two datasets named "player.csv" and "sessions.csv".
### player.csv Variables:
| Variable Name | Description | Type |
| --- | --- | --- |
| experience | Player's experience level | Character |
| subscribe | Whether or not the player is subscribed to a game-related newsletter | Logical |
| hashedEmail | Player's unique hashed email | Character |
| played_hours | Total time the player has played (hours)| Double |
| name | Player's anonymized name | Character |
| gender | Player's gender | Character |
| Age | Player's age (years) | Double |
There are 7 total variables in the "player.csv" dataset, and they were collected through the PLAICraft site when a prospective player self-reports this information before they are able to play for the first time, with the exception for the played_hours variable, which is calculated by adding up the length of all their sessions.

### sessions.csv Variables:
| Variable Name | Description | Type |
| --- | --- | --- |
| hashedEmail | Player's experience level | Character |
| start_time | Time the player's session started (DD/MM/YYYY HH:mm) | Character |
| end_time | Time the player's session ended (DD/MM/YYYY HH:mm) | Character |
| original_start_time | Time the player's session started in Unix time (ms) | Double |
| original_end_time | Time the player's session ended in Unix time (ms) | Double |
There are 5 total variables in the "sessions.csv" dataset; hashedEmail is also collected through the player information form in the PLAICraft website, and the rest of the variables are recorded along with each individual session.

The dataset being used in the project is a combination of the player dataset and the sessions dataset. Each entry is synced up from both datasets with the hashedEmail variable, as that is consistent between the two. The "start_time" and "end_time" variables are subtracted to create a 

## **Chosen Broad Question:** What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
### **Specific Question:** To what accuracy can a player's *age*, *average session length*, and *total play time* predict whether the player is subscribed to a game-related newsletter in the merged player and sessions dataset?